In [1]:
from taiwan_stock_calculator import InvalidPriceError, get_price_range, create_price_table

def color_text(text, color):
    colors = {
        'red': '\033[91m',
        'green': '\033[92m',
        'cyan': '\033[96m',
        'reset': '\033[0m'
    }
    return f"{colors.get(color, '')}{text}{colors['reset']}"

def print_table(headers, rows, column_widths=None):
    if column_widths is None:
        column_widths = [max(len(str(row[i])) for row in rows + [headers]) for i in range(len(headers))]
    
    header_line = "| " + " | ".join(header.ljust(width) for header, width in zip(headers, column_widths)) + " |"
    separator_line = "+" + "+".join("-" * (width + 2) for width in column_widths) + "+"
    
    print(separator_line)
    print(header_line)
    print(separator_line)
    
    for row in rows:
        row_str = "| " + " | ".join(str(item).ljust(width) for item, width in zip(row, column_widths)) + " |"
        print(row_str)
    
    print(separator_line)

def test_get_price_range():
    price_ranges = create_price_table()

    valid_cases = [
        (0.01, 0.01, 0.02),
        (0.02, 0.01, 0.03),
        (4.99, 4.98, 5.00),
        (5, 4.99, 5.01),
        (7.5, 7.49, 7.51),
        (9.99, 9.98, 10.00),
        (10, 9.99, 10.05),
        (25, 24.95, 25.05),
        (50, 49.95, 50.1),
        (75, 74.9, 75.1),
        (100, 99.9, 100.5),
        (125, 124.5, 125.5),
        (150, 149.5, 150.5),
        (300, 299.5, 300.5),
        (500, 499.5, 501.0),
        (750, 749.0, 751.0),
        (1000, 999.0, 1005.0),
        (1500, 1495.0, 1505.0),
        (10000, 9995.0, 10005.0),
    ]

    invalid_cases = [
        0, 0.001, 0.009, -1, -100, "not a number",
        1001, 1234, 9999, 1000000000, float("inf"),
        0.011111, 10.005, 100.0001,
        49.99, 99.99, 149.99, 499.99, 999.99,
    ]

    valid_results = []
    invalid_results = []

    print(color_text("Testing valid cases:", "cyan"))
    for price, expected_down, expected_up in valid_cases:
        try:
            down, up = get_price_range(price, price_ranges)
            status = "Passed" if down == expected_down and up == expected_up else "Failed"
            color = "green" if status == "Passed" else "red"
            valid_results.append([price, expected_down, expected_up, down, up, color_text(status, color)])
        except Exception as e:
            valid_results.append([price, expected_down, expected_up, "-", "-", color_text(f"Failed - {str(e)}", "red")])

    headers = ["Price", "Expected Down", "Expected Up", "Actual Down", "Actual Up", "Status"]
    print_table(headers, valid_results)

    print("\n" + color_text("Testing invalid cases:", "cyan"))
    for price in invalid_cases:
        try:
            get_price_range(price, price_ranges)
            invalid_results.append([price, color_text("Failed - Should have raised an error", "red")])
        except InvalidPriceError as e:
            invalid_results.append([price, color_text(f"Passed - {str(e)}", "green")])
        except Exception as e:
            invalid_results.append([price, color_text(f"Failed - Unexpected error: {str(e)}", "red")])

    headers = ["Price", "Status"]
    print_table(headers, invalid_results)

test_get_price_range()

Testing valid cases:
+-------+---------------+-------------+-------------+-----------+-----------------+
| Price | Expected Down | Expected Up | Actual Down | Actual Up | Status          |
+-------+---------------+-------------+-------------+-----------+-----------------+
| 0.01  | 0.01          | 0.02        | 0.01        | 0.02      | Passed |
| 0.02  | 0.01          | 0.03        | 0.01        | 0.03      | Passed |
| 4.99  | 4.98          | 5.0         | 4.98        | 5.0       | Passed |
| 5     | 4.99          | 5.01        | 4.99        | 5.01      | Passed |
| 7.5   | 7.49          | 7.51        | 7.49        | 7.51      | Passed |
| 9.99  | 9.98          | 10.0        | 9.98        | 10.0      | Passed |
| 10    | 9.99          | 10.05       | 9.99        | 10.05     | Passed |
| 25    | 24.95         | 25.05       | 24.95       | 25.05     | Passed |
| 50    | 49.95         | 50.1        | 49.95       | 50.1      | Passed |
| 75    | 74.9          | 75.1        | 74.9        